In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sbn

sbn.set()

Tasks (SIGURD)

- Create cool plot of busy servers
- Update text and explain erlang distrbution.

<h3>(1) Simulate blocking system</h3>

In [6]:
def simulate_blocking_system(arrival_intensity = 1, mean_service_time = 8, num_servers = 10, n = 10000, 
                             arrival_mode = 'poisson', service_mode = 'exponential', service_params = {}):
    """
        Simulate simple blocking system with discrete events and no waiting room.
    """
    t_system = 0
    m = num_servers
    servers = np.zeros(m)
    blocked = 0 # Counter of number of blocked

    for _ in range(n):
        # Sample time from which this customer arrives
        if arrival_mode == 'poisson':
            t_arrival = stats.expon.rvs(scale=arrival_intensity, size = 1)
        elif arrival_mode == 'erlang':
            t_arrival = stats.erlang.rvs(a = 2, scale=arrival_intensity / 2, size = 1)
        elif arrival_mode == 'hyper':
            t_arrival = rvs_hyperexponential(p = 0.8, lambda_1 = 0.8333, lambda_2 = 5.0)
        else:
            raise ValueError('Wrong arrival mode specified!')
        
        # Extend system time
        t_system += t_arrival

        # Find available server
        min_server_idx = np.argmin(servers)

        if t_system >= servers[min_server_idx]:
            if service_mode == 'exponential':
                t_service = stats.expon.rvs(scale=mean_service_time, size = 1)
            elif service_mode == 'constant':
                t_service = mean_service_time
            elif service_mode == 'pareto':
                k = service_params.get('k')
                t_service = rvs_pareto(mean_ = mean_service_time, k = k, size = 1)
            elif service_mode == 'normal':
                s = service_params.get('s')
                val = stats.norm.rvs(loc = mean_service_time, scale = s, size = 1)
                if val < 0:
                    val = 0
                t_service = val
            else:
                raise ValueError('Wrong service mode specified')
            servers[min_server_idx] = t_system + t_service
        else:
            blocked += 1

    # Compute blocked fraction
    blocked_fraction = blocked / n

    return blocked_fraction

def rvs_hyperexponential(p = 0.8, lambda_1 = 0.83, lambda_2 = 5.0, size = 1):    
    choices = stats.binom.rvs(p = 1 - p, n = 1, size = size)
    results = np.zeros(size)

    for i, choice in enumerate(choices):
        if choice == 0:
            results[i] =  stats.expon.rvs(scale = 1 / lambda_1, size = 1)    
        else:
            results[i] = stats.expon.rvs(scale = 1 / lambda_2, size = 1)

    return results

def rvs_pareto(mean_ = 8, k = 1.05, size = 1):
    # Find the value of Beta
    beta = mean_ * (k - 1) / k

    # Generate uniform numbers 
    U = np.random.uniform(0, 1, size = size)
    X = beta * (U ** (-1/k))

    return X

def confidence_interval(vals, alpha = 0.05):
    if type(vals) != np.ndarray:
        vals = np.array(vals)

    n = len(vals)

    mean_ = np.mean(vals)
    std_error = np.sqrt( 1 / (n - 1) * np.sum((vals - mean_) ** 2))

    t = stats.t.ppf(1 - (alpha / 2), df = n - 1 )

    conf = [mean_ - t * std_error / np.sqrt(n), mean_ + t * std_error / np.sqrt(n)]

    return np.array(conf)

def analytical_blocking_system(arrival_intensity = 1, mean_service_time = 8, num_servers = 10):
    lambda_ = arrival_intensity
    s = mean_service_time
    m = num_servers
    A = lambda_ * s

    temp = np.array([A ** i / np.math.factorial(i) for i in np.arange(0, m + 1, 1)])

    B = (A ** m / np.math.factorial(m)) / (temp.sum())

    return B


Compute analytical solution and compare to the confidence interval of the simulated solution

In [7]:
simulated_fractions = []

for i in range(10):
    blocked_fraction = simulate_blocking_system()
    simulated_fractions.append(blocked_fraction)

conf = confidence_interval(simulated_fractions)

analytical_blocked_fraction = analytical_blocking_system() 

print_conf = conf * 100

print(f'Simulated Blocked fraction confidence interval: [{print_conf[0]:.2f}%, {print_conf[1]:.2f}%]')
print(f'Analytical Blocked fraction: {analytical_blocked_fraction * 100:.2f}%')

Simulated Blocked fraction confidence interval: [11.62%, 12.28%]
Analytical Blocked fraction: 12.17%


The analytical solution is inside the confidence interval of the simulated solution so it seems that the solution is correct

<h3>(2) Experiment with different distributions of arrival time</h3>

In [8]:
def experiment_arrival_time_distributions():
    arrival_modes = ['poisson', 'erlang', 'hyper']

    print('Blocked fraction values of different arrival modes')
    print('-' * 50)
    for mode in arrival_modes:

        rounds = 10

        block_sims = np.zeros(rounds)

        for r in range(rounds):
            block_sims[r] = simulate_blocking_system(arrival_mode=mode)

        conf_int = confidence_interval(block_sims, alpha = 0.05)
        print_conf_int = conf_int * 100

        print(f'arrival mode: {mode}, conf_int = [{print_conf_int[0]:.2f}%, {print_conf_int[1]:.2f}%]')

experiment_arrival_time_distributions()

Blocked fraction values of different arrival modes
--------------------------------------------------
arrival mode: poisson, conf_int = [11.55%, 12.44%]
arrival mode: erlang, conf_int = [8.88%, 9.74%]
arrival mode: hyper, conf_int = [13.73%, 14.37%]


<h3>(3) Experiment with different service-time distributions</h3>

In [5]:
def experiment_service_mode_distributions():
    service_modes = ['exponential', 'constant', 'pareto', 'pareto', 'normal']
    service_params = [{}, {}, {'k' : 1.05}, {'k' : 2.05}, {'s' : 2}]

    print('Blocked fraction values of different service modes')
    print('-' * 50)
    for i, mode in enumerate(service_modes):

        rounds = 10

        block_sims = np.zeros(rounds)

        for r in range(rounds):
            block_sims[r] = simulate_blocking_system(service_mode=mode, service_params=service_params[i])

        conf_int = confidence_interval(block_sims, alpha = 0.05)
        print_conf_int = conf_int * 100

        print(f'service mode: {mode}, params: {service_params[i]}, conf_int = [{print_conf_int[0]:.2f}%, {print_conf_int[1]:.2f}%]')

experiment_service_mode_distributions()

Blocked fraction values of different service modes
--------------------------------------------------
service mode: exponential, params: {}, conf_int = [11.62%, 12.50%]
service mode: constant, params: {}, conf_int = [12.03%, 12.70%]
service mode: pareto, params: {'k': 1.05}, conf_int = [0.01%, 0.13%]
service mode: pareto, params: {'k': 2.05}, conf_int = [12.12%, 13.38%]
service mode: normal, params: {'s': 2}, conf_int = [11.45%, 12.27%]


<h3>(4) Compare differences in confidence intervals for prior tasks</h3>

We start by simulating the program with different arrival- and service-time distributions and compare confidence intervals. Every program is simulated 10 times.

In [9]:
experiment_arrival_time_distributions()
print()
experiment_service_mode_distributions()

Blocked fraction values of different arrival modes
--------------------------------------------------
arrival mode: poisson, conf_int = [11.57%, 12.48%]
arrival mode: erlang, conf_int = [9.05%, 9.67%]
arrival mode: hyper, conf_int = [13.01%, 14.12%]

Blocked fraction values of different service modes
--------------------------------------------------
service mode: exponential, params: {}, conf_int = [11.93%, 12.89%]
service mode: constant, params: {}, conf_int = [11.60%, 12.56%]
service mode: pareto, params: {'k': 1.05}, conf_int = [0.04%, 0.37%]
service mode: pareto, params: {'k': 2.05}, conf_int = [11.65%, 12.12%]
service mode: normal, params: {'s': 2}, conf_int = [11.88%, 12.12%]


The confidence interval for the different arrival time distributions are quite similar, with the hyperexponentiallly distributed arrival times causing slightly larger block fractions than the exponential and erlang. This does make sense as the standard deviation of the arrival times is higher for this distribution, which can lead to congestion in serving the arrivals. The Erlang distribution with $a = 1$ is the same as the exponential distribution, which would make the arrival times identical.

For service modes we get similar results. The block fraction confidence interval is smallest for the constant service time distribution (not counting pareto-distributed service times with k = 1.05), which is to be expected as there is less variability between rounds. Noteably is also that pareto-distributed service times with $k = 1.05$ has the lowest block fraction in general, being almost zero. The mean values of random variables from this distribution is $~2.0$ which is much lower than the mean of the other distributions.